In [1]:
###ARTIST EXAMPLE###
# artist_url = "https://api.spotify.com/v1/artists/"

# pprint(requests.get(artist_url + "66CXWjxzNUsdJxJ2JdwvnR", headers = token_headers).json())

In [2]:
###TRACKS EXAMPLE###
# track_url = "https://api.spotify.com/v1/tracks/"

# pprint(requests.get(track_url + "6ocbgoVGwYJhOv1GgI9NsF", headers = token_headers).json())

In [3]:
###PLAYLIST EXAMPLE###
# playlist_id = "37i9dQZEVXbLRQDuF5jeBp"

# pprint(requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}", headers = token_headers).json())

In [4]:
import requests
import spotipy
from requests.auth import HTTPBasicAuth
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [5]:
from client import client_id, client_secret

In [ ]:
token_url = "https://accounts.spotify.com/api/token"
headers = {'content-type': 'application/x-www-form-urlencoded'}

token_params = {"grant_type" : "client_credentials"}

token_json = requests.post(token_url, params = token_params, headers = headers, auth = (client_id, client_secret)).json()
token = token_json['access_token']

In [ ]:
token_headers = {'authorization': f"Bearer {token}"}

artist_name_list = []
artist_id_list = []
artist_followers_list = []
genre_list = []
genre_popularity = []

search_url = "https://api.spotify.com/v1/search"

print("----------------------\nRetrieving Artist ID\n----------------------")

for artist in top_artist_df["artists"]:
    search_param = {"q" : artist,
                   "type" : "artist",
                   "limit" : 1}
    
    try:
        artist_json = requests.get(search_url, params = search_param, headers = token_headers).json()
        artist_id = artist_json["artists"]["items"][0]["id"]
        artist_name = artist_json["artists"]["items"][0]["name"]
        artist_follower = artist_json["artists"]["items"][0]["followers"]["total"]
        genres = artist_json["artists"]["items"][0]["genres"]
        
        if artist_id not in artist_id_list:
            artist_id_list.append(artist_id)
            artist_name_list.append(artist_name)
            artist_followers_list.append(artist_follower)
            
            for genre in genres:
                if genre not in genre_list:
                    genre_list.append(genre)
                    genre_popularity.append(1)
                    
                else:
                    genre_popularity[genre_list.index(genre)] += 1
                
            print(f"Artist ID Found | {artist_id} | {artist_name}")
            
        else:   
            print("Artist ID in list already")
        
    except IndexError:
        print("Could Not Find Artist ID")

In [ ]:
genre_popularity_df = pd.DataFrame({
    "Genre" : genre_list,
    "Genre Count" : genre_popularity
})

genre_popularity_df["Genre by Artist Percentage"] = (genre_popularity_df["Genre Count"]/len(artist_name_list)) * 100

genre_popularity_df = genre_popularity_df.sort_values(by = "Genre by Artist Percentage", ascending = False).head(15).reset_index()

genre_popularity_df

In [ ]:
fig, ax = plt.subplots(figsize = (40,15))
x_axis = np.arange(15)

ax.bar(x_axis, genre_popularity_df["Genre by Artist Percentage"])
plt.xticks(x_axis, genre_popularity_df["Genre"], Fontsize = 15)
plt.ylabel("Percentage of Artist Genre (%)", Fontsize = 20)
plt.title("Top 50 Artists by Genre 2018", Fontsize = 30)
plt.text(max(x_axis) + 1.3, 60, "Note:\nArtists may have multiple genres.", Fontsize = 20)

rects = ax.patches

for rect in rects:
    height = rect.get_height()
    width = rect.get_x() + 0.4
    if height > 0:
        ax.text(width, height, f'{round(height,2)}%', Fontsize = 20, ha = 'center', va = 'bottom')
    else:
         ax.text(width, height, f'{round(height,2)}%', Fontsize = 20, ha = 'center', va = 'top')

plt.show()